In [1]:
import pandas as pd
import openpyxl
import os
import urllib
import datetime
from db_utils.base import Session, Session_ax
import matplotlib.pyplot as plt
import seaborn as sns
from extract_info import get_armoires_etuves, GetPrice, moteur2demarrer, var2art, map_moteur

In [2]:
def plot_concentric_bubbles(data, x, y, size, hue, max_hue=10):
    """
    Plots a bubble chart with concentric circles.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data.
    x (str): The column name for x-axis values.
    y (str): The column name for y-axis values.
    size (str): The column name for the size of the bubbles.
    hue (str): The column name for the hue of the bubbles.
    """
    # Sort data by hue
    import matplotlib

# Find hues and sort from most to least frequent
    hue_size = data.groupby(hue, as_index=False)[size].sum()
    hue_size = hue_size.sort_values(by=size, ascending=False)
    hue_order = hue_size[hue].values
    # map hue to integer
    hue2int = {hue: i for i, hue in enumerate(hue_order)}
    data['hue_int'] = data[hue].map(hue2int)
    data_sorted = data.sort_values(by='hue_int')

    #  Remove hues with low frequency
    if len(hue_order) > max_hue:
        hue_order = hue_order[:max_hue]
        data_sorted = data_sorted[data_sorted[hue].isin(hue_order)]
        data_sorted['hue_int'] = data_sorted[hue].map(hue2int)

    # Prepare figure
    fig, ax = plt.subplots(1, 1, figsize=(9, 6))

    #  Choose a color palette pastel
    colors = matplotlib.colormaps.get_cmap('Pastel2').colors

    # Iterate over each hue
    for hue_int in range(len(hue_order)):
        hue_value = hue_order[hue_int]
        hue_data = data_sorted[data_sorted['hue_int'] >= hue_int]

        # Cumulative sum for size within each (x, y) group
        to_plot = hue_data.groupby([x, y], as_index=False)[size].sum()
        # Modify size for best plot effect
        to_plot[size] = to_plot[size].apply(lambda x: 100*x**(1))
        # Scatter plot for each hue. linewidth is 0 to remove the line between points
        ax.scatter(to_plot[x], to_plot[y], s=to_plot[size], alpha=1, label=hue_value, linewidth=0, zorder=hue_int, color=colors[hue_int])


    # Calculate total sum of size for each (x, y) group and add as text
    totals = data.groupby([x, y], as_index=False)[size].sum()
    for i, txt in enumerate(totals[size]):
        ax.text(totals[x].iloc[i], totals[y].iloc[i], txt, ha='center', va='center', color='black', zorder=len(hue_order)+1)

    ax.set_xlabel(x)
    ax.set_ylabel(y)
    # Customize and place legend outside the plot
    legend = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    # Make all legend markers the same size
    for handle in legend.legend_handles:
        handle.set_sizes([100])
    return fig, ax

def plot_bubble_price(data, x, y, size, label):
    """
    Plots a bubble chart with prices

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data.
    x (str): The column name for x-axis values.
    y (str): The column name for y-axis values.
    size (str): The column name for the size of the bubbles.
    hue (str): The column name for the annotation
    """
    # Sort data by hue
    import matplotlib

    # Prepare figure
    fig, ax = plt.subplots(1, 1, figsize=(9, 9))

    #  Choose a color palette pastel
    colors = matplotlib.colormaps.get_cmap('Pastel2').colors
    ax.scatter(data[x], data[y], s=data[size]*50, alpha=1, linewidth=0, color=colors[0])
    # Add label as annotation in the plot
    for i, txt in enumerate(data[label]):
        ax.text(data[x].iloc[i], data[y].iloc[i], txt, ha='center', va='center', color='black', zorder=len(data)+1)
    return fig, ax

In [3]:
df_etuve = get_armoires_etuves(cache=False)

# On garde uniquememnt les étuves réconnues en tant que telles sur Divalto
# df_etuve = df_etuve[df_etuve['type_equip_divalto'] == 'ETUVE']

adv_classes = ["ADV-ETUVE-MOD-BT", "ADV-ETUVE-MOD-HT", "ADV-ETUVE-MONO"]
df_etuve = df_etuve[df_etuve['ADV'].isin(adv_classes)]
# Remove variateur
# Strip column Dem_Mot
df_etuve.loc[:, 'Dem_Mot'] = df_etuve['Dem_Mot'].str.strip()
df_etuve = df_etuve[~df_etuve['Dem_Mot'].str.contains('ariate')]
print(f"Nombre d'étuves: {len(df_etuve)}")
# Show repartition of the different types of etuves
df_etuve.groupby('ADV', as_index=False)['Num_AF'].count()



Nombre d'étuves: 190


,ADV,Num_AF
0,ADV-ETUVE-MOD-BT,62
1,ADV-ETUVE-MOD-HT,63
2,ADV-ETUVE-MONO,65


In [4]:
distinct_cases

NameError: name 'distinct_cases' is not defined

In [ ]:
# On essaye de filtrer les monoblocs:
df = df_etuve[df_etuve['ADV'] == 'ADV-ETUVE-MOD-HT'].copy()
#  Remove where price is 0
df = df[df['armoire_price'] > 0]
# Replace no ME1 by 0
df['ME1'] = df['ME1'].fillna(0)

# Filter by Regul
df = df[df['Regul'].str.lower().str.strip() == 'fuji pxf4']
#  Filter by chauffage
df = df[df['Chauff'].str.lower().str.strip() == "bruleur à apport d'air"]

distinct_cases = df.groupby(['MS1', 'ME1'], as_index=False).agg(
    {'Num_AF': 'count', 'armoire_price': ['mean', 'std', 'max', 'min']}
    )


# fig, ax = plot_concentric_bubbles(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'Chauff')

distinct_cases['mean_price'] = distinct_cases['armoire_price']['mean'].apply(lambda x: round(x, -1)).astype(int)
distinct_cases['std_label'] = distinct_cases['armoire_price']['std'].apply(lambda x: f"\n+/- {x:.0f}" if not pd.isna(x) else '')
distinct_cases['label'] = distinct_cases['mean_price'].astype(str) + distinct_cases['std_label'].astype(str)

fig, ax = plot_bubble_price(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'label')



In [ ]:
# On essaye de filtrer les monoblocs:
df = df_etuve[df_etuve['ADV'] == 'ADV-ETUVE-MOD-HT'].copy()
#  2 moteurs de soufflage
df = df[df['Nb_MS'] == 2]

# Get nomber of armoire for each cluster
distinct_cases = df.groupby(['MS1', 'ME1'], as_index=False)['Num_AF'].count()
# Join with df
df = df.merge(distinct_cases, on=['MS1', 'ME1'], how='left', suffixes=('', '_count'))
#  Remove where price is 0
df = df[df['armoire_price'] > 0]
# Replace no ME1 by 0
df['ME1'] = df['ME1'].fillna(0)

# Make a dummy field from Regul (there are many different values)

from sklearn.preprocessing import OneHotEncoder
# Add a column with year
df['year'] = df['Date'].dt.year

# Extract size
df[['Length', 'Width', 'Height']] = df['Armoire'].apply(extract_dimensions)

#  Remove rows where dimensions are not available
df = df[~df['Length'].isna()]

df['volume'] = df['Length'] * df['Width'] * df['Height']

print(f"Nbre d'étuves: {len(df)}")

# Create an instance of the OneHotEncoder

enc1 = OneHotEncoder(handle_unknown='ignore')
# Apply the encoder to the df
OH_Reg = enc1.fit_transform(df['Regul'].str.lower().str.strip().values.reshape(-1, 1)).toarray()
#  Print alternative categories
print(enc1.categories_)
enc2 = OneHotEncoder(handle_unknown='ignore')
OH_Chauff = enc2.fit_transform(df['Chauff'].str.lower().str.strip().values.reshape(-1, 1)).toarray()
#  Print alternative categories
print(enc2.categories_)






# Train a linear regression with lasso to infer the price of the armoire
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_validate
import numpy as np

# Use a k-fold cross validation to find the best alpha

y = df['armoire_price'].values

X = df[['volume']].values
X = np.concatenate((X, df[['MS1', 'ME1']].values), axis=1)
# Add OH encoding
X = np.concatenate((X, OH_Reg), axis=1)
X = np.concatenate((X, OH_Chauff), axis=1)
X = np.concatenate((X, df['year'].values.reshape(-1, 1)), axis=1)
# X = np.concatenate((X, df['Nb_Cab_Ident'].values.reshape(-1, 1)), axis=1)
X = np.concatenate((X, df['Num_AF_count'].values.reshape(-1, 1)), axis=1)
# X = np.concatenate((X, df[['Length', 'Width', 'Height']].values), axis=1)

#  Normalize data
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
y_std = np.std(y)
y = (y - np.mean(y)) / y_std


#  Model is lasso with intercept
model = Lasso(fit_intercept=True, alpha=0.05)

# Model is knn
from sklearn.neighbors import KNeighborsRegressor
# model = KNeighborsRegressor(n_neighbors=5)

# Take just Dummy regressor with mean
from sklearn.dummy import DummyRegressor
# model = DummyRegressor(strategy='mean')

# Define a leave 2 out cross validation
from sklearn.model_selection import LeavePOut
cv = LeavePOut(p=3)

# Perform cross-validation
cv_results = cross_validate(model, X, y, cv=cv, return_estimator=True, scoring='neg_mean_squared_error')

# Extract R2 scores
nmse_scores = cv_results['test_score']

# Extract coefficients
coefficients = np.array([estimator.coef_ for estimator in cv_results['estimator']])

# Calculate the mean and standard deviation of the coefficients
mean_coefficients = np.mean(coefficients, axis=0)
std_coefficients = np.std(coefficients, axis=0)

# Print results
mean_mse = np.mean(-nmse_scores)
print("Mean MSE:", mean_mse)
r2 = 1 - mean_mse / np.var(y)
print("R2:", r2)
print("Mean coefficients:", mean_coefficients)
print("Standard deviation of coefficients:", std_coefficients)
mean_err_euros = np.abs(mean_mse)**0.5 * y_std
print(f"Mean absolute error (euros): {mean_err_euros:.0f}")






In [ ]:
df['Armoire']

def extract_dimensions(dim_str):
    none_return = pd.Series([None, None, None], index=['Length', 'Width', 'Height'])
    if pd.isna(dim_str):
        return none_return
    dim_str = dim_str.strip().lower()
    parts = dim_str.split('x')
    if len(parts) == 3:
        try:
            L, l, H = map(int, parts)
            return pd.Series([L, l, H], index=['Length', 'Width', 'Height'])
        except ValueError:
            # Handle case where conversion to int fails
            return none_return
    else:
        # Handle case where the format is not LxlxH
        return none_return


# Apply the function to the dimensions column and join the result to the original DataFrame
df[['Length', 'Width', 'Height']] = df['Armoire'].apply(extract_dimensions)
df

In [ ]:
df

In [ ]:
y

In [ ]:
# On essaye de filtrer les monoblocs:
df = df_etuve[df_etuve['ADV'] == 'ADV-ETUVE-MONO'].copy()
#  Remove where price is 0
df = df[df['armoire_price'] > 0]
# Replace no ME1 by 0
df['ME1'] = df['ME1'].fillna(0)
distinct_cases = df.groupby(['MS1', 'ME1', 'Chauff'], as_index=False)['Num_AF'].count()

fig, ax = plot_concentric_bubbles(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'Chauff')
tot_mono = sum(distinct_cases['Num_AF'])
fig.suptitle(f'Distribution des puissances moteurs pour les étuves monoblocs (TOT: {tot_mono})')
plt.show()



In [ ]:
modul = df_etuve[df_etuve['ADV'] == "ADV-ETUVE-MOD-BT"].copy()
# Print statistics for Nb_MS
modul.groupby('Nb_MS', as_index=False)['Num_AF'].count()


In [ ]:
# LOW TEMPERATURE - 1 MOTOR
df = df_etuve[df_etuve['ADV'] == "ADV-ETUVE-MOD-BT"].copy()

# Replace no ME1 by 0
df['ME1'] = df['ME1'].fillna(0)
# 1 seul moteur de soufflage
df = df[df['Nb_MS'] == 1]

# distinct_cases = df.groupby(['MS1', 'ME1', 'Chauff'], as_index=False)['Num_AF'].count()
distinct_cases = df.groupby(['MS1', 'ME1', 'Dem_Mot'], as_index=False)['Num_AF'].count()
#  Plot
# fig, ax = plot_concentric_bubbles(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'Chauff')
fig, ax = plot_concentric_bubbles(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'Dem_Mot')
tot = sum(distinct_cases['Num_AF'])
fig.suptitle(f'Distribution des puissances moteurs pour les étuves modulaires BT (1 mot souffl.) (TOT: {tot})')
plt.show()


In [ ]:
# LOW TEMPERATURE - 2 MOTOR
df = df_etuve[df_etuve['ADV'] == "ADV-ETUVE-MOD-BT"].copy()

# Replace no ME1 by 0
df['ME1'] = df['ME1'].fillna(0)
# 1 seul moteur de soufflage
df_1M = df[df['Nb_MS'] == 2]

# distinct_cases = df_1M.groupby(['MS1', 'ME1', 'Chauff'], as_index=False)['Num_AF'].count()
distinct_cases = df_1M.groupby(['MS1', 'ME1', 'Dem_Mot'], as_index=False)['Num_AF'].count()
#  Plot
# fig, ax = plot_concentric_bubbles(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'Chauff')
fig, ax = plot_concentric_bubbles(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'Dem_Mot')
tot = sum(distinct_cases['Num_AF'])
fig.suptitle(f'Distribution des puissances moteurs pour les étuves modulaires BT (2 mot souffl.) (TOT: {tot})')
plt.show()


In [ ]:
# HIGH TEMPERATURE - 2 MOTOR
df = df_etuve[df_etuve['ADV'] == "ADV-ETUVE-MOD-HT"].copy()

# Replace no ME1 by 0
df['ME1'] = df['ME1'].fillna(0)
# 1 seul moteur de soufflage
df = df[df['Nb_MS'] == 2]

distinct_cases = df.groupby(['MS1', 'ME1', 'Chauff'], as_index=False)['Num_AF'].count()
# distinct_cases = df.groupby(['MS1', 'ME1', 'Regul'], as_index=False)['Num_AF'].count()
# distinct_cases = df.groupby(['MS1', 'ME1', 'Dem_Mot'], as_index=False)['Num_AF'].count()
#  Plot
fig, ax = plot_concentric_bubbles(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'Chauff')
# fig, ax = plot_concentric_bubbles(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'Regul')
tot = sum(distinct_cases['Num_AF'])
fig.suptitle(f'Distribution des puissances moteurs pour les étuves modulaires HT (2 mot souffl.) (TOT: {tot})')
plt.show()


In [ ]:
columns = ['Code_Equip', 'Date', 'Caract', 'Nb_MS', 'MS1', 'MS2', 'ME1',
           'Dem_Mot', 'Chauff', 'Regul', 'ADV', 'puissance_bruleur', 'qty_bruleur', 'armoire_price']

#  COnvert ot int, including NAN
df_etuve = get_armoires_etuves(cache=False)
df_etuve = df_etuve.loc[:, columns].copy()
df_etuve['armoire_price'] = df_etuve['armoire_price'].fillna(0).astype(int)

# Export to xls
filename = f'./temp/etuve.xlsx'
writer = pd.ExcelWriter(filename, engine='openpyxl')

# MOD-HT
df = df_etuve[df_etuve['ADV'] == "ADV-ETUVE-MOD-HT"].copy()
# Create a sheet 'MOD-HT'
writer.sheets['MOD-HT'] = df.to_excel(writer, sheet_name='MOD-HT', index=False)

# MOD-BT
df = df_etuve[df_etuve['ADV'] == "ADV-ETUVE-MOD-BT"].copy()
# Create a sheet 'MOD-BT'
writer.sheets['MOD-BT'] = df.to_excel(writer, sheet_name='MOD-BT', index=False)

# MONO
df = df_etuve[df_etuve['ADV'] == "ADV-ETUVE-MONO"].copy()
# Create a sheet 'MONO'
writer.sheets['MONO'] = df.to_excel(writer, sheet_name='MONO', index=False)

# Remaining
df = df_etuve[~df_etuve['ADV'].isin(["ADV-ETUVE-MOD-HT", "ADV-ETUVE-MOD-BT", "ADV-ETUVE-MONO"])].copy()
# Create a sheet 'MONO'

writer.sheets['OTHER'] = df.to_excel(writer, sheet_name='OTHER', index=False)

writer.close()




In [ ]:
df
# 3 Population
# 1. Monobloc
# 2. Modulaire 1 moteur soufflage
# 3. Modulaire 2 moteurs soufflage

# Essayer de récupérer code ADV sur AX.
#  Baasse température -> ADV-ETUVE-MOD-BT et 1 moteur
# Haute températue -> ADV-ETUVE-MOD-HT et 2 moteurs
# Monobloc -> ADV-ETUVE-MONO (on exclue par l'instant)
#  Le reste est du spécial

# Regarder si on peut récupérer la puissance /type bruleur (sous-ensemble I)

# Exclure variateurs

# Refaire l'analyse avec le prix moyen de l'armoire, vérifier l'ecart
# Trier les liste avec Julien G. pour vérifier

# Faire tourner avec 1,2,3 standards sur les 3 populations



# Exclure AF004086-A

In [ ]:
df = modul[modul['Nb_MS'] == 1]
# df = df[df['Chauff'].str.lower().str.strip() == 'bva 3 points']
# Apply strip on the column Regul
df.loc[:, 'Regul'] = df['Regul'].str.strip()
distinct_cases = df.groupby(['MS1', 'ME1', 'Regul'], as_index=False)['Num_AF'].count()

fig, ax = plot_concentric_bubbles(distinct_cases, 'MS1', 'ME1', 'Num_AF', 'Regul')
tot = sum(distinct_cases['Num_AF'])
fig.suptitle(f'Distribution des puissances moteurs pour les étuves modulaires (2 mot souffl.) (TOT: {tot})')
plt.show()

In [ ]:
from spec_equipement import build_tree, convert_tree_to_json_list_info
import json

project_code = 'AF003650-A'
df, tree =build_tree(project_code)
ADV = df[df['code_article'].str.contains('ADV')]['code_article'].values[0]
ADV

json_tree = convert_tree_to_json_list_info(tree)
# Dumps as file
with open(f'./temp/{project_code}_aslistinfo.json', 'w') as f:
    f.write(json.dumps(json_tree, indent=4))